In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from roboflow import Roboflow
import os
from PIL import Image


In [ ]:
rf = Roboflow(api_key="OpP5CXPumS9XFTDUBooC")
project = rf.workspace("bennett-univ").project("full-body-gender-classification-dataset")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Full-Body-Gender-Classification-Dataset-1 in yolov5pytorch:: 100%|██████████| 6578/6578 [00:02<00:00, 2452.65it/s]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import ViTForImageClassification, ViTFeatureExtractor
from roboflow import Roboflow
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.


In [ ]:
from transformers import ViTFeatureExtractor

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
import os
from PIL import Image

class GenderDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]  # Assuming images are in .jpg format
        self.labels = [0 if "female" in f else 1 for f in self.image_files]  # Assuming labels are derived from filenames (male/female)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(img_name).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torch.utils.data import DataLoader

# Create Dataset instances for training and validation
train_dataset = GenderDataset(data_dir=os.path.join("/content/Full-Body-Gender-Classification-Dataset-1/train/images"), transform=transform)
val_dataset = GenderDataset(data_dir=os.path.join("/content/Full-Body-Gender-Classification-Dataset-1/valid/images"), transform=transform)

# Create DataLoader instances for training and validation
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)


In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,  # Adjust for binary classification (male/female)

    ignore_mismatched_sizes=True  # Ignore mismatched sizes for classifier layer
)
'''attn_implementation="sdpa",
    torch_dtype=torch.float16,'''

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'attn_implementation="sdpa",\n    torch_dtype=torch.float16,'

In [ ]:
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Set up device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_dataloader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")


100%|██████████| 180/180 [01:48<00:00,  1.66it/s]


Epoch 1/3, Loss: 0.0006545254456644518


100%|██████████| 180/180 [01:50<00:00,  1.63it/s]


Epoch 2/3, Loss: 1.8574372757029777e-05


100%|██████████| 180/180 [01:52<00:00,  1.60it/s]

Epoch 3/3, Loss: 8.765214303391783e-06


In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(val_dataloader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.4f}")


100%|██████████| 18/18 [00:04<00:00,  3.79it/s]

Validation Accuracy: 1.0000


In [ ]:
# Save the trained model
model.save_pretrained("./vit_gender_model")


In [ ]:
from transformers import ViTForImageClassification

# Load the trained model
model = ViTForImageClassification.from_pretrained("./vit_gender_model")
model.to(device)  # Ensure the model is on the same device as the training (CPU or GPU)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
from PIL import Image

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    # Apply the same transformations as during training
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Example usage
image_path = "/content/test1.jpg"  # Replace with your image path
input_image = preprocess_image(image_path)


In [ ]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    inputs = input_image.to(device)  # Move the image to the same device as the model (CPU or GPU)
    outputs = model(inputs)
    logits = outputs.logits  # Get the raw predictions

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(logits, dim=1)

# Get the predicted class
predicted_class_id = torch.argmax(probabilities, dim=1).item()
gender_label = "Male" if predicted_class_id == 1 else "Female"

print(f"Predicted Gender: {gender_label}")
print(f"Confidence Score: {probabilities[0][predicted_class_id].item():.4f}")


Predicted Gender: Male
Confidence Score: 0.9997
